# Test Misc Functions
This notebook contains snippets of code used to test various functions. 

Some functions take the source data as input to perform operations. This assumes that the data processing has been done, please check paths.

Some functions generate output files, which are stored in the folder data/samples/<some name>. If this does not exist, an error will be thrown, generate this folder first.

# Test Edge Detection on Depth
This is used to generate training data for the outlines model. The outlines model has updated code.

In [ ]:
%matplotlib inline 

import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from data_loader import Dataset,Options
import OpenEXR, Imath
from scipy.misc import imsave

class OPT():
    def __init__(self):
        self.dataroot = './data/'
        self.file_list = './data/datalist'
        self.batchSize = 32
        self.shuffle = True
        self.phase = 'train'
        self.num_epochs = 500
        self.imsize = 224
        self.num_classes = int(3)
        self.gpu = '0'
        self.logs_path = 'logs/exp9'
        self.use_pretrained = False

opt = OPT()

###################### DataLoader #############################
dataloader = Dataset(opt)
display_output = True
save_output = False

depth_path = 'data/source-files/depth-imgs/%09d-depth.exr'
edges_output_path = 'data/samples/edges-depth-imgs/%09d-depth-edges.jpg'

if not (os.path.isdir(os.path.dirname(depth_path))):
    raise ValueError('The folder %s does not exist. Please check path.'%(os.path.dirname(depth_path)))
if(save_output):
    if not (os.path.isdir(os.path.dirname(edges_output_path))):
        raise ValueError('The folder %s does not exist. Please check path.'%(os.path.dirname(edges_output_path)))


for i in range (1,100):
    # Load Depth Img and Apply Blur
    print('Loading img %d'%(i))
    depth_img_orig = dataloader.exr_loader(depth_path%(i), ndim=1)
    depth_img_orig = cv2.GaussianBlur(depth_img_orig,(5,5),0)
    
    # Make all depth values greater than 2.5m as 0 (for masking edge matrix)
    depth_img_mask = depth_img_orig.copy()
    depth_img_mask[depth_img_mask > 2.5]=0
    depth_img_mask[depth_img_mask > 0 ]=1
    
    # Apply Laplacian filters for edge detection
    edges_lap = cv2.Laplacian(depth_img_orig, cv2.CV_8U, ksize=7, borderType=0 )
    edges_lap_binary = edges_lap.copy()
    edges_lap_binary[edges_lap_binary>1] = 255
    edges_lap_binary[edges_lap_binary<=1] = 0
    edges_lap_binary = edges_lap_binary * depth_img_mask
    
    # Apply Canny filter
    depth_img_masked = depth_img_orig * depth_img_mask
    depth_img_masked *= 255.0/depth_img_masked.max()
    depth_img_masked = np.uint8(depth_img_masked)
    edges = cv2.Canny(depth_img_masked,5,10)
    
    
    if(display_output):
        fig1 = plt.figure(figsize=(12,12))
        plt.title('Original Depth Image')
        plt.imshow(depth_img_orig, cmap='gray')
        plt.show()
        fig2 = plt.figure(figsize=(12,12))
        plt.title('Laplacian Filter applied to Depth Image')
        plt.imshow(edges_lap, cmap='gray')
        plt.show()
        fig3 = plt.figure(figsize=(12,12))
        plt.title('Laplacian Filter Converted to Binary')
        plt.imshow(edges_lap_binary, cmap='gray')
        plt.show()
#         fig4 = plt.figure(figsize=(12,12))
#         plt.title('Canny Filter Applied to Depth Image')
#         plt.imshow(edges, cmap='gray')
#         plt.show()
    
    if(save_output):
        imsave( edges_output_path%(i), edges_lap_binary )



# Test Edge Detection on Surface Normals
This is used to generate training data for the outlines model. The outlines model has updated code.

In [ ]:
%matplotlib inline 

import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from data_loader import Dataset,Options
import OpenEXR, Imath
from scipy.misc import imsave
import imageio

class OPT():
    def __init__(self):
        self.dataroot = './data/'
        self.file_list = './data/datalist'
        self.batchSize = 32
        self.shuffle = True
        self.phase = 'train'
        self.num_epochs = 500
        self.imsize = 224
        self.num_classes = int(3)
        self.gpu = '0'
        self.logs_path = 'logs/exp9'
        self.use_pretrained = False

opt = OPT()

###################### DataLoader #############################
dataloader = Dataset(opt)
save_output = False

depth_path_rgb = 'data/train/preprocessed-camera-normals/rgb-visualizations/%09d-cameraNormals.png'
edges_output_path = 'data/samples/edges-normals-imgs/%09d-normals-edges.jpg'

if not (os.path.isdir(os.path.dirname(depth_path_rgb))):
    raise ValueError('The folder %s does not exist. Please check path.'%(os.path.dirname(depth_path)))
if(save_output):
    if not (os.path.isdir(os.path.dirname(edges_output_path))):
        raise ValueError('The folder %s does not exist. Please check path.'%(os.path.dirname(edges_output_path)))

for i in range (1,100):
    # Load Depth Img and Apply Blur
    print('Loading img %d'%(i))
    depth_img_orig_rgb = imageio.imread(depth_path_rgb%(i))
    
    
    # Apply Laplacian filters for edge detection
    # TODO: CONVERT TO HSV INSTEAD, OR COMPUTE GRADIENT ON EACH CHANNEL SEPARATELY
    depth_img_orig = cv2.cvtColor(depth_img_orig_rgb, cv2.COLOR_BGR2GRAY)
    depth_img_orig = cv2.GaussianBlur(depth_img_orig,(5,5),0)
    edges_lap = cv2.Laplacian(depth_img_orig, cv2.CV_8U, ksize=3, borderType=1 )
    
    # Convert Laplacian output to binary for training data
    edges_lap_binary = edges_lap.copy()
    edges_lap_binary[edges_lap_binary>15] = 255
    edges_lap_binary[edges_lap_binary<=10] = 0
    
    
    display_output = 1
    if(display_output):
        fig1 = plt.figure(figsize=(12,12))
        plt.title('Surface Normals RGB Representation')
        plt.imshow(depth_img_orig_rgb)
        plt.show()
        fig2 = plt.figure(figsize=(12,12))
        plt.title('Laplacian Filter Applied to Surface Normals')
        plt.imshow(edges_lap, cmap='gray')
        plt.show()
        fig3 = plt.figure(figsize=(12,12))
        plt.title('Laplacian Filter Converted to Binary')
        plt.imshow(edges_lap_binary, cmap='gray')
        plt.show()
    
    if(save_output):
        imsave( edges_output_path%(i), edges_lap_binary )



# Generating HDF5 .h5 files
This is used to convert the output of the surface normal prediction model to suitable input format for the depth2depth module. 

In [ ]:
import h5py
import numpy as np
import os

### Reading a file
path_h5 = 'data/depth2depth_sample_files/normal_scannet_realsense_test/realsense_004_normal_est.h5'
path_h5_test = 'data/samples/surface_normal_hdf5/mytestfile.h5'
h5_key = 'result'

if not (os.path.isdir(os.path.dirname(path_h5))):
    raise ValueError('The folder %s does not exist. Please check path.'%(os.path.dirname(depth_path)))
if not (os.path.isdir(os.path.dirname(path_h5_test))):
    raise ValueError('The folder %s does not exist. Please check path.'%(os.path.dirname(edges_output_path)))


print('Reading hdf5 file \'{}\' ...\n'.format(path_h5))
f = h5py.File(path_h5, 'r')

f_keys = list(f.keys())
print('f_keys:', f_keys)

dset = f[h5_key]
print('dset.shape:', dset.shape)
print('dset.dtype:', dset.dtype)
print('dset.name:', dset.name)
print('---')

# Reading values from a HDF5 dataset
dset_data = dset[()]
print('datatype in dataset: ', type(dset_data))

### Writing a file
with h5py.File(path_h5_test, "w") as f:
    dset2 = f.create_dataset('/result', data=dset_data)
    

### Reading back contents of written file
with h5py.File(path_h5_test, 'r') as f:
    f_keys = list(f.keys())
    print('List keys: ', f_keys)
    dset2 = f[h5_key]
    dset2_data = dset2[:]

if np.array_equal(dset_data, dset2_data):
    print('2 arrays equal')
else:
    print('2 arrays not equal')

